In [1]:
from haversine import haversine, Unit
import os, sys
from math import sqrt, cos, pi

In [2]:
KM_PER_DEGREE_LAT = 111  # Approximate km per degree of latitude
def euclidean(from_location, to_location):
    lat1, lon1 = from_location
    lat2, lon2 = to_location

    # Calculate km per degree of longitude based on average latitude
    avg_lat = (lat1 + lat2) / 2
    km_per_degree_lon = KM_PER_DEGREE_LAT * cos(avg_lat * pi / 180)  # converting degrees to radians

    # Convert degree differences to kilometers
    x = (lon2 - lon1) * km_per_degree_lon
    y = (lat2 - lat1) * KM_PER_DEGREE_LAT
    return sqrt(x**2 + y**2)

In [3]:
# Helper function to compute distance between a list of coordinates (Trajectory distance)
# Haversine distance used
def calculate_trajectory_distance(positions: list[tuple[float]]) -> float:
    """
    Calculate the trajectory distance for a trajectory

    :param: List of coordinates (lat, lon)

    :return: Float (km) -> Combined distance between all pairs of points in km
    """
    distance = 0
    for i in range(1, len(positions)):
        from_location = positions[i - 1]
        to_location = positions[i]

        # distance += haversine(from_location, to_location, unit=Unit.KILOMETERS)
        distance += euclidean(from_location, to_location)
    return distance

In [4]:
# Read files and calculate trajectory distances
def process_trajectories(directory, prefix='K'):
    distances = []
    for filename in os.listdir(directory):
        if filename.startswith(prefix):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                coordinates = []
                for line in file:
                    parts = line.strip().split(',')
                    if len(parts) == 2:
                        lat, lon = float(parts[0]), float(parts[1])
                        coordinates.append((lat, lon))
                if coordinates:
                    distances.append(calculate_trajectory_distance(coordinates))
    if distances:
        average_distance = sum(distances) / len(distances)
    else:
        average_distance = 0
    return average_distance

In [7]:
currentdir = os.path.dirname(os.path.abspath("__file__"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
from constants import ROME_OUTPUT_FOLDER,  PORTO_OUTPUT_FOLDER


directory_path = f"../{ROME_OUTPUT_FOLDER}"
average_distance = process_trajectories(directory_path, 'K')
print(f"The average trajectory distance is: {average_distance:.2f} km")

The average trajectory distance is: 13.76 km
